In [34]:
# # Clear output folder
# import os

# def remove_folder_contents(folder):
#     for the_file in os.listdir(folder):
#         file_path = os.path.join(folder, the_file)
#         try:
#             if os.path.isfile(file_path):
#                 os.unlink(file_path)
#             elif os.path.isdir(file_path):
#                 remove_folder_contents(file_path)
#                 os.rmdir(file_path)
#         except Exception as e:
#             print(e)

# folder_path = '/kaggle/working'
# remove_folder_contents(folder_path)
# os.rmdir(folder_path)

In [35]:
# To remove a file
# os.remove("/kaggle/working/file_name.csv")

In [36]:
# !pip download tabpfn --no-deps -d pip-packages
# !pip install tabpfn
# from tabpfn import TabPFNClassifier
# TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')

# !mv /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt pip-packages/
# !zip -r pip-packages.zip pip-packages

In [37]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/packages-icr/pip-packages

Looking in links: file:///kaggle/input/packages-icr/pip-packages


In [38]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [39]:
import numpy as np
import pandas as pd
import json
import plotly.express as px
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer 
from catboost import Pool, CatBoostClassifier
import xgboost
from tabpfn import TabPFNClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

In [40]:
# LOAD THE DATA

BASE_DIR = '/kaggle/input/icr-identify-age-related-conditions'
train_file = f'{BASE_DIR}/train.csv'
greeks_file = f'{BASE_DIR}/greeks.csv'
test_file = f'{BASE_DIR}/test.csv'

train_df = pd.read_csv(train_file)
greeks_df = pd.read_csv(greeks_file)
test_df = pd.read_csv(test_file)

print(train_df.EJ.unique())

['B' 'A']


In [42]:
#Take out the test 
test_log_loss = train_df[:120]
train_log_loss = train_df.iloc[::-1].iloc[:-120][::-1]
greeks_log_loss = greeks_df.iloc[::-1].iloc[:-120][::-1]
train_log_loss.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
120,30202b353d8a,55.0,2988915.0,9783.0,1796.0,1552.0,530.0,42.0,1168.0,175.0,395496.0,35.0,42293.0,24838.0,150679.0,60331.0,3529.0,74558.0,10254.0,2199.0,44.0,167.0,7330.0,6916.0,242.0,76420.0,15948.5,30722.0,680.0,153.0,43545.0,51365.0,8034.0,180.0,772.0,3203.0,1189.0,521.0,190277.0,23.0,B,92762.0,12378.0,756.0,10697.0,798.0,1454499.0,1522.0,4.939967,857.0,10.0,932.0,34303.0,2585182.0,3132.0,11282.0,0.379500,0
121,3026974712cd,267.0,681505.5,19562.0,19228.0,2011.0,984.0,248.0,1630.0,1188.5,1017354.0,72.0,68029.0,15610.0,188394.5,507576.0,11737.0,133186.0,10010.0,168.0,76.0,165.0,11165.0,14323.5,243.0,237097.0,8475.0,40233.0,2462.0,153.0,39374.5,54597.0,8504.0,7015.0,673.0,4719.0,2315.5,330.0,206928.0,188.0,B,84486.0,21983.5,9127.0,11260.0,3021.0,2334952.0,1941.0,14.283000,2632.5,84.0,2225.0,16439.0,3382783.5,11856.0,13109.0,0.079595,0
122,3101fca6b743,122.0,2344915.5,13769.5,11213.0,1552.0,323.0,42.0,2488.0,175.0,917925.0,54.0,45902.5,3983.0,76447.0,60331.0,5041.5,124672.0,9227.0,2944.0,104.0,192.0,12722.0,20853.5,168.0,402009.0,8094.0,67034.5,680.0,111.0,35633.5,30505.0,7330.0,45.0,673.0,14271.0,811.5,1491.5,374627.5,17.0,B,144814.0,8472.0,866.5,9218.0,303.0,668588.5,2260.0,1.268857,985.5,75.0,3486.5,17510.0,85458.0,13009.0,29928.0,1.122000,0
123,312661adb2ff,157.0,586307.5,14272.0,5243.0,2058.0,518.0,274.0,2016.0,1530.5,670052.0,47.0,51748.0,2071.0,94068.5,181814.5,6877.0,113972.0,7568.0,2441.5,54.0,252.0,5784.0,19982.0,75.0,491664.0,11252.5,113212.0,3902.0,69.0,57870.0,25758.0,5666.0,50.0,673.0,866.0,1575.5,1124.0,337174.0,17.0,B,254371.0,33368.5,9699.0,9920.0,950.0,763569.5,1118.0,4.193459,7257.5,10.0,2063.0,16439.0,3695584.0,3448.0,9114.0,1.009800,0
124,319f7e210021,103.0,605985.0,14228.0,1551.0,1552.0,573.0,42.0,1211.0,175.0,376414.0,69.0,51574.0,3998.0,177218.0,60331.0,1360.0,108079.5,6187.0,228.0,83.0,165.0,6093.0,10513.0,186.0,413936.0,8824.0,95698.0,680.0,135.0,24062.0,46082.0,6800.0,1850.0,673.0,7321.0,697.0,525.0,214205.0,54.0,B,74173.0,8472.0,756.0,12411.0,1102.0,1264356.0,1866.0,9.315077,1873.0,10.0,3269.0,16439.0,198963.0,9046.0,6006.0,0.086692,0


In [43]:
first_category = train_df.EJ.unique()[0]
train_df.EJ = train_df.EJ.eq(first_category).astype('int')
test_df.EJ = test_df.EJ.eq(first_category).astype('int')

# first_category = train_log_loss.EJ.unique()[0]
# train_log_loss.EJ = train_log_loss.EJ.eq(first_category).astype('int')
# test_log_loss.EJ = test_log_loss.EJ.eq(first_category).astype('int')

In [44]:
from datetime import datetime
times = greeks_df.Epsilon.copy()
times[greeks_df.Epsilon != 'Unknown'] = greeks_df.Epsilon[greeks_df.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeks_df.Epsilon == 'Unknown'] = np.nan

# times = greeks_log_loss.Epsilon.copy()
# times[greeks_log_loss.Epsilon != 'Unknown'] = greeks_log_loss.Epsilon[greeks_log_loss.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
# times[greeks_log_loss.Epsilon == 'Unknown'] = np.nan

In [45]:
target_column = 'Class'
predictor_columns = [n for n in train_df.columns if n != target_column and n != 'Id']
# predictor_columns = [n for n in train_log_loss.columns if n != target_column and n != 'Id']

In [46]:
class WeightedEnsemble(BaseEstimator):
    def __init__(self):
        self.classifiers = [xgboost.XGBClassifier(), TabPFNClassifier(N_ensemble_configurations=64, device='cuda:0')]
#         self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        self.imputer = IterativeImputer(max_iter=10, random_state=0)
    
    def fit(self, X, y):
        unique_classes, y = np.unique(y, return_inverse=True)
        self.classes_ = unique_classes
        X = self.imputer.fit_transform(X)
        for classifier in self.classifiers:
            classifier.fit(X, y)
    
    def predict_proba(self, X):
        X = self.imputer.transform(X)
        probabilities = np.stack([classifier.predict_proba(X) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)

In [47]:
train_pred_and_time = pd.concat((train_df[predictor_columns], times), axis=1)
test_predictors = np.array(test_df[predictor_columns])
test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

# train_pred_and_time = pd.concat((train_log_loss[predictor_columns], times), axis=1)
# test_predictors = np.array(test_log_loss[predictor_columns])
# test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

In [48]:
model = WeightedEnsemble()
model.fit(np.array(train_pred_and_time), np.array(greeks_df['Alpha']))

# model.fit(np.array(train_pred_and_time), np.array(greeks_log_loss['Alpha']))

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [49]:
probabilities = model.predict_proba(test_pred_and_time)
assert (model.classes_[0] == 'A')
probabilities = np.concatenate((probabilities[:,:1], np.sum(probabilities[:,1:], 1, keepdims=True)), axis=1)

In [50]:
p0 = probabilities[:,:1]
p0[p0 > 0.86] = 1
p0[p0 < 0.14] = 0

In [51]:
def competition_log_loss(y_true,y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    y_pred = y_pred.T[1]
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    sum_0 = -np.sum((1 - y_true) * np.log(p_0))
    sum_1 = -np.sum(y_true * np.log(p_1))
    to_output = ((1 / N_0) * sum_0 + (1 / N_1) * sum_1) / 2

    return to_output

In [52]:
# # redefine p0 to calculate log_loss
# a = [np.array(_).item() for _ in 1-p0]
# a = pd.DataFrame(a)

In [53]:
# print(competition_log_loss(test_log_loss['Class'], a))

In [54]:
submission = pd.DataFrame(test_df["Id"], columns=["Id"])

# submission = pd.DataFrame(test_log_loss["Id"], columns=["Id"])
submission["class_0"] = p0
submission["class_1"] = 1 - p0
submission.to_csv('submission.csv', index=False)

In [58]:
submission_df = pd.read_csv('submission.csv')
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


In [56]:
# submission_df['True_class'] = test_log_loss['Class']
# for index, row in submission_df.iterrows():
#     if (row[1] == 1 and row[3] == 0) or (row[2] == 1 and row[3] == 1):
#         submission_df = submission_df.drop(index)
        
# submission_df

,Id,class_0,class_1,True_class
0,00eed32682bb,0.5,0.5,1
1,010ebe33f668,0.5,0.5,0
2,02fa521e1838,0.5,0.5,0
3,040e15f562a2,0.5,0.5,0
4,046e85c7cc7f,0.5,0.5,1


In [57]:
# submission_df.count()